In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="org-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125214
Azure region: southcentralus
Subscription id: a7e82b92-bbed-4497-a8df-009b71b94f9c
Resource group: aml-quickstarts-125214


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "pkd-compute"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
from azureml.train.estimator import Estimator
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.50,1.00),
       "--max_iter" : choice(10,20,30)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.',
                 entry_script = 'train.py',
                 compute_target =  cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = ps,
                                   primary_metric_name = 'Accuracy',
                                   max_total_runs = 50,
                                   max_concurrent_runs = 4,
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                   policy = policy,
                                   estimator = est)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()


print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])


Best Run Id:  HD_ea88245e-00c8-4c42-bc0c-91dd314fdacf_1

 Accuracy: 0.9072837632776934


In [12]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_b7bf034210c536e361ba2d29198ccce3ab8f5a3b29a8835653dfa0f588739800_d.txt',
 'azureml-logs/65_job_prep-tvmps_b7bf034210c536e361ba2d29198ccce3ab8f5a3b29a8835653dfa0f588739800_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_b7bf034210c536e361ba2d29198ccce3ab8f5a3b29a8835653dfa0f588739800_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/118_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds= TabularDatasetFactory.from_delimited_files(path=url)

In [4]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [5]:
import pandas as pd
y_dataframe = pd.DataFrame(y,columns=['y'])
y_dataframe.head()

,y
0,0
1,0
2,0
3,0
4,0


In [6]:
cleanedAllData = pd.concat([x,y_dataframe],axis =1)
cleanedAllData.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(cleanedAllData,train_size=0.80, test_size=0.20, shuffle=True)

In [12]:
x_train.to_csv("training/autotrain.csv")

In [13]:
defaultds= ws.get_default_datastore()
print(defaultds)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-1f417f78-1ef5-42f1-93da-84eac5d975bf",
  "account_name": "mlstrg125214",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


In [14]:
defaultds.upload(src_dir='training/',target_path='bankmarketing',overwrite= True, show_progress=True)

Uploading an estimated of 1 files
Uploading training/autotrain.csv
Uploaded training/autotrain.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_033b7cf16bec4c15b2735a28c5989e84

In [15]:
from azureml.core import Dataset
train_data = Dataset.Tabular.from_delimited_files(path=defaultds.path('bankmarketing/autotrain.csv'))
print(train_data)

TabularDataset
{
  "source": [
    "('workspaceblobstore', 'bankmarketing/autotrain.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}


In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
label_column_name = 'y'
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name=label_column_name,
    compute_target =  cpu_cluster,
    n_cross_validations=4)

In [17]:
# Submit your automl run
remote_run = exp.submit(automl_config, show_output = True)

Running on remote.
Running on remote compute: pkd-compute
Parent Run ID: AutoML_b4016165-b8d4-4621-ad68-d0441de47457

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--

In [19]:
# Retrieve and save your best automl model.

best_model, fitted_model = remote_run.get_output()
print(best_model)
print(fitted_model)


Run(Experiment: org-project,
Id: AutoML_b4016165-b8d4-4621-ad68-d0441de47457_29,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(allow_chargram=None, enable_dnn=None,
                                 enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedso...
                                                                                                  l1_ratio=0.26530612244897955,
                                                                                                  learnin

In [ ]:
cpu_cluster.delete()